### Baixando imagens GEFS no site do ONS
##O obejtivo desse script é automatizar o processo de baixar imagens do site do ONS.


In [6]:
#Imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import datetime
import os
import time
import shutil
import os.path

In [7]:
#O objetivo dessa parte é organizar o salvamento de imagens 

#Essa parte verifica se já existe um diretório com o nome do dia em questao (pois as imagens serão salvas em uma pasta com esse nome)
hoje= datetime.date.today()
path = ('/Users/railtonsabtos/Desktop/ONS/GEFS/{}'.format(str(hoje)))

#Caso a pasta já exista, ela será deletada 
isdir = os.path.isdir(path)  
if isdir == True:
   shutil.rmtree(path)

#Caso não exista, esse diretório com o nome do dia será criado para salvar as imagens   
os.mkdir('GEFS/{}'.format(str(hoje)))

In [8]:
#Chamando o programa
#Indo de 1 a 11, ou seja 10 interações

i = 0
for i in range(1,11):
    navegation(i) 

A imagem 1o foi salva com sucesso!
A imagem 2o foi salva com sucesso!
A imagem 3o foi salva com sucesso!
A imagem 4o foi salva com sucesso!
A imagem 5o NAO foi salva com sucesso!
A imagem 5o NAO foi salva com sucesso!
A imagem 5o foi salva com sucesso!
A imagem 6o foi salva com sucesso!
A imagem 7o foi salva com sucesso!
A imagem 8o NAO foi salva com sucesso!
A imagem 8o NAO foi salva com sucesso!
A imagem 8o NAO foi salva com sucesso!
A imagem 8o foi salva com sucesso!
A imagem 9o foi salva com sucesso!
A imagem 10o foi salva com sucesso!


In [5]:
# O real programa

def navegation(i):
    
    #Usando o Webdriver do Chrome
    options = webdriver.ChromeOptions()
    #No modo oculto
    options.add_argument("--headless")
    options.add_argument("--disable-gpu") 
    navegador = webdriver.Chrome(options=options)
    #O Robo irá navegar até o endereço indicado, onde a url varia a cada interação 
    url = 'https://sintegre.ons.org.br/sites/9/38/Documents/images/operacao_integrada/meteorologia/global/glob50_{}.gif'.format(i)
    navegador.get(url)
    
    #O Site SinIntegra do ONS necessita de login e senha
    login = 'Usuario'
    senha = 'Senha'
    
    #será colocado o login e a senha nos espacos indicados e será apertado Enter
    navegador.find_element_by_id('username').send_keys(login)
    navegador.find_element_by_id('username').send_keys(Keys.ENTER)
    time.sleep(2)
    
    navegador.find_element_by_id('password').send_keys(senha)
    navegador.find_element_by_id('password').send_keys(Keys.ENTER)
    
    #tempo de espera para pagina carregar.
    timeout = 10    
    time.sleep(3)
               
    try:
        element_present = EC.presence_of_element_located((By.XPATH , '/html/body/img'))
        WebDriverWait(navegador, timeout).until(element_present)
        img = navegador.find_element_by_xpath('/html/body/img')
        url = img.get_attribute('src')
   
        # Recupera os cookies da sessÃ£o do chrome pelo driver do selenium
        all_cookies = navegador.get_cookies()
        # Cria um mapa para aramzenar todos os cookies (chave:valor)
        cookies = {}
        # Popula o mapa com os cookies
        for s_cookie in all_cookies:
            cookies[s_cookie["name"]]=s_cookie["value"]
   
        #utiliza a biblioteca requests passando a url da imagem e os cookies de autenticaÃ§Ã£o
        response = requests.get(url, cookies=cookies)
        hoje= datetime.date.today()
        file_name = "GEFS/{}/GEFS_{}_{}.png".format(str(hoje),i, hoje)
        if response.status_code == 200:
                # Se a requst ocorreu bem, salvar o binario

            with open(file_name, 'wb') as f:
                f.write(response.content)
        #Indicação que a imagem foi salva com sucesso        
        print("A imagem {}o foi salva com sucesso!".format(i))
        i = i+1
         
        navegador.quit()
        
        #no caso de erro (de DDoS ou de tempo de carregamento) será usado a excessao de TimeoutException
    except TimeoutException:
        
        #Indicaçao que a imagem nao foi salva
        print("A imagem {}o NAO foi salva com sucesso!".format(i))
        #A propria função será chamada novamente em uma tentativa de tentar até conseguir hehe (Solução recusiva)
        navegation(i)
        navegador.quit()